In [20]:
# Importing dataset
import os
from os import listdir
from os import path
from os.path import isfile, join, getsize
import pandas as pd
import numpy as np
from pycaret.classification import *
import seaborn as sns
import pickle

In [21]:
os.chdir('/home/jupyter-chandler.vaughn/data/')
pairs = pd.read_pickle('cover80_song_pair_data.pkl')
pairs_stem = pd.read_pickle('cover80_song_pair_stem_data.pkl')

#plagarisms = pd.read_pickle('/volumes/CCV_SSD_1/similar_songs/plagerism_song_pair_data.pkl')
#plagarisms_stem = pd.read_pickle('/volumes/CCV_SSD_1/similar_songs/plagerism_song_pair_stem_data.pkl')

#song_random = pd.read_pickle('cover80_song_whitenoise_data.pkl')

In [22]:
merge_pairs = pd.merge(pairs, pairs_stem, on=['pair'])
merge_pairs = merge_pairs.drop(['song_1_y', 'song_2_y'], axis = 1)
merge_pairs = merge_pairs[merge_pairs['pair'] != 'All_Along_The_Watchtower']
merge_pairs = merge_pairs[merge_pairs['pair'] != 'Let_It_Be']
merge_pairs = merge_pairs[merge_pairs['pair'] != 'Take_Me_To_The_River']

In [23]:
merge_pairs = merge_pairs.drop(['MFCC_1', 
                                'MFCC_2', 
                                'GFCC_1', 
                                'GFCC_2', 
                                'HPCP_1', 
                                'HPCP_2', 
                                'sim_matrix',
                                'sim_matrix_bass',
                                'sim_matrix_drums',
                                'sim_matrix_vocals',
                                'sim_matrix_other',
                                'MFCC_1_bass',
                               'MFCC_1_drums',        
                               'MFCC_1_vocals',          
                               'MFCC_1_other',             
                               'GFCC_1_bass',               
                               'GFCC_1_drums',           
                               'GFCC_1_vocals',          
                               'GFCC_1_other',        
                               'HPCP_1_bass',            
                               'HPCP_1_drums',            
                               'HPCP_1_vocals',          
                               'HPCP_1_other',             
                               'MFCC_2_bass',           
                               'MFCC_2_drums',         
                               'MFCC_2_vocals',        
                               'MFCC_2_other',          
                               'GFCC_2_bass',           
                               'GFCC_2_drums',          
                               'GFCC_2_vocals',       
                               'GFCC_2_other',        
                               'HPCP_2_bass',           
                               'HPCP_2_drums',              
                               'HPCP_2_vocals',      
                               'HPCP_2_other'], axis=1)

In [24]:
merge_pairs['sim_distance'] = np.log(merge_pairs['sim_distance']+0.0001)
merge_pairs['sim_distance_bass'] = np.log(merge_pairs['sim_distance_bass']+0.0001)
merge_pairs['sim_distance_argmin'] = np.log(merge_pairs['sim_distance_argmin']+0.0001)
merge_pairs['sim_distance_drums'] = np.log(merge_pairs['sim_distance_drums']+0.0001)
merge_pairs['sim_distance_vocals'] = np.log(merge_pairs['sim_distance_vocals']+0.0001)
merge_pairs['sim_distance_other'] = np.log(merge_pairs['sim_distance_other']+0.0001)

merge_pairs['sim_distance_argmin_combined'] = merge_pairs[['sim_distance','sim_distance_argmin']].min(axis=1)

#remove inf pairs
merge_pairs = merge_pairs[~merge_pairs.isin([np.nan, np.inf, -np.inf]).any(1)]
merge_pairs['target'] = 1

In [27]:
merge_pairs = merge_pairs.rename(columns = {'song_1_x':'song_1', 'song_2_x':'song_2'})

In [28]:
merge_pairs.head()

,pair,song_1,song_2,sim_distance,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin,sim_distance_argmin_combined,target
0,A_Whiter_Shade_Of_Pale,annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.mp3,procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of...,-3.853485,-3.119863,-3.274098,-3.323253,-3.785736,-3.785736,-3.853485,1
1,Abracadabra,steve_miller_band+Steve_Miller_Band_Live_+09-A...,sugar_ray+14_59+11-Abracadabra.mp3,-3.752329,-3.396255,-3.318984,-3.726317,-3.307305,-3.726317,-3.752329,1
2,Addicted_To_Love,robert_palmer+Riptide+03-Addicted_To_Love.mp3,tina_turner+Tina_Live_In_Europe_CD_1_+09-Addic...,-4.035278,-3.568458,-3.468068,-3.276847,-3.837415,-3.837415,-4.035278,1
6,All_Tomorrow_s_Parties,velvet_underground+Andy_Warhol_with_Nico_+06-A...,japan+Quiet_Life+06-All_Tomorrow_s_Parties.mp3,-3.382072,-3.023531,-3.356503,-3.849179,-2.812766,-3.849179,-3.849179,1
7,America,simon_and_garfunkel+Bookends+03-America.mp3,paul_simon+Concert_in_the_Park_Disc_2+09-Ameri...,-3.877601,-4.010184,-3.278129,-3.577138,-3.629846,-4.010184,-4.010184,1


In [40]:
random_stem = pd.read_pickle('cover80_song_random_stem_data.pickle')

In [41]:
random_stem.columns

Index(['pair', 'song_1', 'song_2', 'MFCC_1_bass', 'MFCC_1_drums',
       'MFCC_1_vocals', 'MFCC_1_other', 'GFCC_1_bass', 'GFCC_1_drums',
       'GFCC_1_vocals', 'GFCC_1_other', 'HPCP_1_bass', 'HPCP_1_drums',
       'HPCP_1_vocals', 'HPCP_1_other', 'MFCC_2_bass', 'MFCC_2_drums',
       'MFCC_2_vocals', 'MFCC_2_other', 'GFCC_2_bass', 'GFCC_2_drums',
       'GFCC_2_vocals', 'GFCC_2_other', 'HPCP_2_bass', 'HPCP_2_drums',
       'HPCP_2_vocals', 'HPCP_2_other', 'sim_matrix_bass', 'sim_matrix_drums',
       'sim_matrix_vocals', 'sim_matrix_other', 'sim_distance_bass',
       'sim_distance_drums', 'sim_distance_vocals', 'sim_distance_other',
       'sim_distance_argmin', 'MFCC_1', 'GFCC_1', 'HPCP_1', 'MFCC_2', 'GFCC_2',
       'HPCP_2', 'sim_matrix', 'sim_distance'],
      dtype='object')

In [42]:
random_stem = random_stem.drop(['MFCC_1', 
                                'MFCC_2', 
                                'GFCC_1', 
                                'GFCC_2', 
                                'HPCP_1', 
                                'HPCP_2', 
                                'sim_matrix',
                                'sim_matrix_bass',
                                'sim_matrix_drums',
                                'sim_matrix_vocals',
                                'sim_matrix_other',
                                'MFCC_1_bass',
                               'MFCC_1_drums',        
                               'MFCC_1_vocals',          
                               'MFCC_1_other',             
                               'GFCC_1_bass',               
                               'GFCC_1_drums',           
                               'GFCC_1_vocals',          
                               'GFCC_1_other',        
                               'HPCP_1_bass',            
                               'HPCP_1_drums',            
                               'HPCP_1_vocals',          
                               'HPCP_1_other',             
                               'MFCC_2_bass',           
                               'MFCC_2_drums',         
                               'MFCC_2_vocals',        
                               'MFCC_2_other',          
                               'GFCC_2_bass',           
                               'GFCC_2_drums',          
                               'GFCC_2_vocals',       
                               'GFCC_2_other',        
                               'HPCP_2_bass',           
                               'HPCP_2_drums',              
                               'HPCP_2_vocals',      
                               'HPCP_2_other'], axis=1)

In [43]:
random_stem['sim_distance'] = np.log(random_stem['sim_distance']+0.0001)
random_stem['sim_distance_bass'] = np.log(random_stem['sim_distance_bass']+0.0001)
random_stem['sim_distance_argmin'] = np.log(random_stem['sim_distance_argmin']+0.0001)
random_stem['sim_distance_drums'] = np.log(random_stem['sim_distance_drums']+0.0001)
random_stem['sim_distance_vocals'] = np.log(random_stem['sim_distance_vocals']+0.0001)
random_stem['sim_distance_other'] = np.log(random_stem['sim_distance_other']+0.0001)

random_stem['sim_distance_argmin_combined'] = random_stem[['sim_distance','sim_distance_argmin']].min(axis=1)

#remove inf pairs
random_stem = random_stem[~random_stem.isin([np.nan, np.inf, -np.inf]).any(1)]
random_stem['target'] = 0

In [44]:
random_stem.head()

,pair,song_1,song_2,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin,sim_distance,sim_distance_argmin_combined,target
0,random,everly_brothers+The_Fabulous_Style_of+01-Claud...,queen+Sheer_Heart_Attack+08-Stone_Cold_Crazy,-2.355505,-2.925685,-2.446600,-2.708545,-2.925685,-2.845544,-2.925685,0
0,random,green_day+Kerplunk+16-My_Generation,big_star+Radio_City+10-September_Gurls,-2.933756,-3.089581,-2.422822,-2.444963,-3.089581,-2.772278,-3.089581,0
0,random,beatles+Revolver+14-Tomorrow_Never_Knows,garth_brooks+The_Chase+05-Walking_After_Midnight,-2.280153,-3.084989,-2.369852,-2.312978,-3.084989,-2.399897,-3.084989,0
0,random,creedence_clearwater_revival+Live_in_Europe+08...,en_vogue+Funky_Divas+09-Yesterday,-2.786141,-3.012903,-2.895848,-2.665697,-3.012903,-3.021898,-3.021898,0
0,random,queen+Sheer_Heart_Attack+08-Stone_Cold_Crazy,persuaders+Thin_Line_Between_Love_Hate_Golden_...,-1.881601,-2.963285,-2.255226,-2.709486,-2.963285,-2.633830,-2.963285,0


In [45]:
white = pd.read_pickle('cover80_song_whitenoise_data.pkl')
white_stem = pd.read_pickle('cover80_song_whitenoise_stem_data.pkl')

In [46]:
white.head(2)

,pair,song_1,song_2,MFCC_1,GFCC_1,HPCP_1,MFCC_2,GFCC_2,HPCP_2,sim_matrix,sim_distance
0,A_Whiter_Shade_Of_Pale,annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.mp3,0.guassian-whitenoise-0db.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0306772e-05, 0.00027894697, 5.8507256e-05,...","[[0.0008410609, 0.018806035, 0.0057406384, 0.0...","[[0.8140233, 0.43358523, 0.3934993, 0.15289561...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.145042
1,A_Whiter_Shade_Of_Pale,procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of...,0.guassian-whitenoise-0db.wav,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0306772e-05, 0.00027894697, 5.8507256e-05,...","[[0.0008410609, 0.018806035, 0.0057406384, 0.0...","[[0.8140233, 0.43358523, 0.3934993, 0.15289561...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.106851


In [47]:
white_stem.head(2)

,pair,song_1,song_2,MFCC_1_bass,MFCC_1_drums,MFCC_1_vocals,MFCC_1_other,GFCC_1_bass,GFCC_1_drums,GFCC_1_vocals,...,HPCP_2_other,sim_matrix_bass,sim_matrix_drums,sim_matrix_vocals,sim_matrix_other,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin
0,A_Whiter_Shade_Of_Pale,annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale,0.guassian-whitenoise-0db,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[0.7308082, 0.7091959, 1.0, 0.2683892, 0.2410...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.067277,0.048368,0.057709,0.122656,0.048368
1,Abracadabra,steve_miller_band+Steve_Miller_Band_Live_+09-A...,0.guassian-whitenoise-0db,"[[5.4035194e-09, 1.0683395e-18, 0.0, 0.0, 0.0,...","[[9.842001e-05, 1.594722e-13, 4.281959e-13, 2....","[[5.0497794e-05, 1.5254655e-13, 5.655497e-13, ...","[[9.675984e-06, 2.35649e-13, 9.3634695e-14, 2....","[[4.250656e-07, 8.417094e-17, 0.0, 0.0, 0.0, 5...","[[0.0077416375, 1.4810904e-11, 2.8574804e-11, ...","[[0.0039726105, 1.45000244e-11, 5.1205897e-11,...",...,"[[0.7308082, 0.7091959, 1.0, 0.2683892, 0.2410...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.100649,0.045179,0.049756,0.181359,0.045179


In [48]:
white_pairs = pd.merge(white, white_stem, on=['pair'])
white_pairs = white_pairs.drop(['song_1_y', 'song_2_y'], axis = 1)

In [49]:
white_pairs = white_pairs.drop(['MFCC_1', 
                                'MFCC_2', 
                                'GFCC_1', 
                                'GFCC_2', 
                                'HPCP_1', 
                                'HPCP_2', 
                                'sim_matrix',
                                'sim_matrix_bass',
                                'sim_matrix_drums',
                                'sim_matrix_vocals',
                                'sim_matrix_other',
                                'MFCC_1_bass',
                               'MFCC_1_drums',        
                               'MFCC_1_vocals',          
                               'MFCC_1_other',             
                               'GFCC_1_bass',               
                               'GFCC_1_drums',           
                               'GFCC_1_vocals',          
                               'GFCC_1_other',        
                               'HPCP_1_bass',            
                               'HPCP_1_drums',            
                               'HPCP_1_vocals',          
                               'HPCP_1_other',             
                               'MFCC_2_bass',           
                               'MFCC_2_drums',         
                               'MFCC_2_vocals',        
                               'MFCC_2_other',          
                               'GFCC_2_bass',           
                               'GFCC_2_drums',          
                               'GFCC_2_vocals',       
                               'GFCC_2_other',        
                               'HPCP_2_bass',           
                               'HPCP_2_drums',              
                               'HPCP_2_vocals',      
                               'HPCP_2_other'], axis=1)

In [50]:
white_pairs['sim_distance'] = np.log(white_pairs['sim_distance']+0.0001)
white_pairs['sim_distance_bass'] = np.log(white_pairs['sim_distance_bass']+0.0001)
white_pairs['sim_distance_argmin'] = np.log(white_pairs['sim_distance_argmin']+0.0001)
white_pairs['sim_distance_drums'] = np.log(white_pairs['sim_distance_drums']+0.0001)
white_pairs['sim_distance_vocals'] = np.log(white_pairs['sim_distance_vocals']+0.0001)
white_pairs['sim_distance_other'] = np.log(white_pairs['sim_distance_other']+0.0001)

white_pairs['sim_distance_argmin_combined'] = white_pairs[['sim_distance','sim_distance_argmin']].min(axis=1)

#remove inf pairs
white_pairs = white_pairs[~white_pairs.isin([np.nan, np.inf, -np.inf]).any(1)]
white_pairs['target'] = 0

In [52]:
white_pairs = white_pairs.rename(columns = {'song_1_x':'song_1', 'song_2_x':'song_2'})

In [53]:
white_pairs.head(2)

,pair,song_1,song_2,sim_distance,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin,sim_distance_argmin_combined,target
0,A_Whiter_Shade_Of_Pale,annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.mp3,0.guassian-whitenoise-0db.wav,-1.930046,-2.697455,-3.02686,-2.850604,-2.097559,-3.02686,-3.02686,0
1,A_Whiter_Shade_Of_Pale,procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of...,0.guassian-whitenoise-0db.wav,-2.235382,-2.697455,-3.02686,-2.850604,-2.097559,-3.02686,-3.02686,0


In [57]:
merged_training = pd.concat([merge_pairs,random_stem,white_pairs], ignore_index=True)

In [61]:
merged_training.head()

,pair,song_1,song_2,sim_distance,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin,sim_distance_argmin_combined,target
0,A_Whiter_Shade_Of_Pale,annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.mp3,procol_harum+Greatest_Hits+2-A_Whiter_Shade_Of...,-3.853485,-3.119863,-3.274098,-3.323253,-3.785736,-3.785736,-3.853485,1
1,Abracadabra,steve_miller_band+Steve_Miller_Band_Live_+09-A...,sugar_ray+14_59+11-Abracadabra.mp3,-3.752329,-3.396255,-3.318984,-3.726317,-3.307305,-3.726317,-3.752329,1
2,Addicted_To_Love,robert_palmer+Riptide+03-Addicted_To_Love.mp3,tina_turner+Tina_Live_In_Europe_CD_1_+09-Addic...,-4.035278,-3.568458,-3.468068,-3.276847,-3.837415,-3.837415,-4.035278,1
3,All_Tomorrow_s_Parties,velvet_underground+Andy_Warhol_with_Nico_+06-A...,japan+Quiet_Life+06-All_Tomorrow_s_Parties.mp3,-3.382072,-3.023531,-3.356503,-3.849179,-2.812766,-3.849179,-3.849179,1
4,America,simon_and_garfunkel+Bookends+03-America.mp3,paul_simon+Concert_in_the_Park_Disc_2+09-Ameri...,-3.877601,-4.010184,-3.278129,-3.577138,-3.629846,-4.010184,-4.010184,1


In [62]:
os.chdir('/home/jupyter-chandler.vaughn/data/')
with open('merged_training.pickle', 'wb') as f:
    pickle.dump(merged_training, f)

In [64]:
plagerism = pd.read_pickle('plagerism_song_pair_data.pkl')
plagerism_stem = pd.read_pickle('plagerism_song_pair_stem_data.pkl')

In [65]:
plagerism_pairs = pd.merge(plagerism, plagerism_stem, on=['pair'])
plagerism_pairs = plagerism_pairs.drop(['song_1_y', 'song_2_y'], axis = 1)

In [66]:
plagerism_pairs = plagerism_pairs.drop(['MFCC_1', 
                                'MFCC_2', 
                                'GFCC_1', 
                                'GFCC_2', 
                                'HPCP_1', 
                                'HPCP_2', 
                                'sim_matrix',
                                'sim_matrix_bass',
                                'sim_matrix_drums',
                                'sim_matrix_vocals',
                                'sim_matrix_other',
                                'MFCC_1_bass',
                               'MFCC_1_drums',        
                               'MFCC_1_vocals',          
                               'MFCC_1_other',             
                               'GFCC_1_bass',               
                               'GFCC_1_drums',           
                               'GFCC_1_vocals',          
                               'GFCC_1_other',        
                               'HPCP_1_bass',            
                               'HPCP_1_drums',            
                               'HPCP_1_vocals',          
                               'HPCP_1_other',             
                               'MFCC_2_bass',           
                               'MFCC_2_drums',         
                               'MFCC_2_vocals',        
                               'MFCC_2_other',          
                               'GFCC_2_bass',           
                               'GFCC_2_drums',          
                               'GFCC_2_vocals',       
                               'GFCC_2_other',        
                               'HPCP_2_bass',           
                               'HPCP_2_drums',              
                               'HPCP_2_vocals',      
                               'HPCP_2_other'], axis=1)

In [67]:
plagerism_pairs['sim_distance'] = np.log(plagerism_pairs['sim_distance']+0.0001)
plagerism_pairs['sim_distance_bass'] = np.log(plagerism_pairs['sim_distance_bass']+0.0001)
plagerism_pairs['sim_distance_argmin'] = np.log(plagerism_pairs['sim_distance_argmin']+0.0001)
plagerism_pairs['sim_distance_drums'] = np.log(plagerism_pairs['sim_distance_drums']+0.0001)
plagerism_pairs['sim_distance_vocals'] = np.log(plagerism_pairs['sim_distance_vocals']+0.0001)
plagerism_pairs['sim_distance_other'] = np.log(plagerism_pairs['sim_distance_other']+0.0001)

plagerism_pairs['sim_distance_argmin_combined'] = plagerism_pairs[['sim_distance','sim_distance_argmin']].min(axis=1)

#remove inf pairs
plagerism_pairs = plagerism_pairs[~plagerism_pairs.isin([np.nan, np.inf, -np.inf]).any(1)]
plagerism_pairs = plagerism_pairs.rename(columns = {'song_1_x':'song_1', 'song_2_x':'song_2'})

In [68]:
plagerism_pairs.head()

,pair,song_1,song_2,sim_distance,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin,sim_distance_argmin_combined
0,1,1.Juice WRLD - Lucid Dreams_Cpy.mp3,1.yellowcard-holly-wood-died_Org.mp3,-3.001165,-2.961422,-3.291261,-2.981874,-2.977280,-3.291261,-3.291261
1,10,10.bruno-mars-uptown-funk_Cpy.mp3,10.the-gap-band-oops-up-side-your-head_Org.mp3,-3.130112,-2.983658,-3.323097,-3.200124,-2.573021,-3.323097,-3.323097
2,11,11.drake-who-do-you-love-feat_Cpy.mp3,11.rappin-4-tay-playaz-club_Org.mp3,-3.114593,-3.021577,-3.274226,-3.318774,-2.753680,-3.318774,-3.318774
3,12,12.the-doors-hello-i-love-you_Cpy.mp3,12.the-kinks-all-day-and-all-of-the-night_Org.mp3,-2.953844,-2.921536,-3.036883,-2.442322,-3.098474,-3.098474,-3.098474
5,14,14.breakbot-baby-i-m-yours_Org.mp3,14.bruno-mars-treasure_Cpy.mp3,-3.279673,-1.731434,-3.445135,-3.127537,-3.144894,-3.445135,-3.445135


In [69]:
os.chdir('/home/jupyter-chandler.vaughn/data/')
with open('plagerism_unseen.pickle', 'wb') as f:
    pickle.dump(plagerism_pairs, f)